<a href="https://colab.research.google.com/github/vruddhis/sem6/blob/main/compilerDesign/dag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from typing import Dict, List, Tuple
import graphviz

dot = graphviz.Digraph('expression')

class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.label = str(id(self))

        if left is None and right is None:
            self.leaf = True
        if left is not None:
            dot.edge(self.label, left.label)
        if right is not None:
            dot.edge(self.label, right.label)
        self.left = left
        self.right = right
        dot.node(self.label, str(self.val))
        print("Made new node ", self.val)

def find_parantheses(expression: str) -> Tuple[int, int]:
    stack = []
    for index, char in enumerate(expression):
        if char == '(':
            stack.append(index)
        elif char == ')':
            if stack:
                start_index = stack.pop()
                if not stack:
                    return (start_index, index)
    return None

already_computed: Dict[str, Node] = {}

def make_dag(expression: str) -> Node:
    subtrees: List[Node] = []

    while (paren_indices := find_parantheses(expression)) is not None:
        start_index, end_index = paren_indices
        subtrees.append(make_dag(expression[start_index + 1: end_index]))
        expression = expression[:start_index] + str(len(subtrees) - 1) + expression[end_index + 1:]
        print(expression)

    operators = ['/', '*', '+', '-']
    for op in operators:
        while op in expression:
            index = expression.find(op)

            left_index = index - 1
            if expression[left_index].isdigit():
               while left_index >= 0 and expression[left_index].isdigit():
                    left_index -= 1
               left_index += 1

            right_index = index + 1
            if expression[right_index].isdigit():
               while right_index < len(expression) and expression[right_index].isdigit():
                  right_index += 1
               right_index -= 1

            left_part = expression[left_index:index]
            right_part = expression[index + 1:right_index + 1]

            print(left_part, right_part)
            if left_part.isdigit():
                left_child = subtrees[int(left_part)]
            else:
                if left_part not in already_computed:
                    already_computed[left_part] = Node(left_part)
                left_child = already_computed[left_part]

            if right_part.isdigit():
                right_child = subtrees[int(right_part)]
            else:
                if right_part not in already_computed:
                    already_computed[right_part] = Node(right_part)
                right_child = already_computed[right_part]

            if left_part + op + right_part in already_computed:
                root = already_computed[left_part + op + right_part]
            else:
                root = Node(val=op, left=left_child, right=right_child)
                already_computed[left_part + op + right_part] = root

            subtrees.append(root)
            expression = expression[:left_index] + str(len(subtrees) - 1) + expression[right_index + 1:]
            print(expression)

    if expression.isdigit():
        return subtrees[int(expression)]
    else:
        if expression not in already_computed:
            already_computed[expression] = Node(expression)
        return already_computed[expression]


#okay so this works great but what's with the plus without any parents?????????????


In [24]:
expression = "(a+b+c)/b+c*a-d+(a+b+c)"
tree = make_dag(expression)

a b
Made new node  a
Made new node  b
Made new node  +
0+c
0 c
Made new node  c
Made new node  +
1
0/b+c*a-d+(a+b+c)
a b
0+c
0 c
1
0/b+c*a-d+1
0 b
Made new node  /
2+c*a-d+1
c a
Made new node  *
2+3-d+1
2 3
Made new node  +
4-d+1
d 1
Made new node  d
Made new node  +
4-5
4 5
Made new node  -
6


In [25]:
dot.render('dag', format='png', view=True)

'dag.png'